In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import time
from absl import app
from absl import flags
import numpy as np
import tensorflow as tf


from tensorflow.contrib.tpu.python.tpu import async_checkpoint
from tensorflow.contrib.training.python.training import evaluation
from tensorflow.core.protobuf import rewriter_config_pb2
from tensorflow.python.estimator import estimator
from tensorflow.python.keras import backend as K

from operator import itemgetter
from heapq import nlargest

import sys
sys.path.append('../nas-search')
import search_main, imagenet_input
from search_main import nas_model_fn



In [2]:

FLAGS = flags.FLAGS
FLAGS(["data_format, input_image_size, num_label_classes, num_eval_images, eval_batch_size, num_train_images, train_batch_size, use_tpu"])
flags.FLAGS.data_format="channels_last"
flags.FLAGS.input_image_size = 64
flags.FLAGS.num_label_classes =200
#flags.FLAGS.num_label_classes =10
flags.FLAGS.num_eval_images=9832
flags.FLAGS.eval_batch_size=1024
flags.FLAGS.num_train_images=100000
flags.FLAGS.train_batch_size=1024
flags.FLAGS.use_tpu=False

# Hyperparameters

In [3]:
model_dir = "/Users/roxanefischer/Desktop/models_tiny_imagenet_v2"
data_dir = "/Users/roxanefischer/Documents/cours/3A/Stage_ML/single-path-nas/other/tiny_imagenet/all"
ckpt ="/Users/roxanefischer/Desktop/models_tiny_imagenet_v2/model.ckpt-10000"
#ckpt="/Users/roxanefischer/Desktop/cifar/models/model.ckpt-6"

save_checkpoints_steps=10000
log_step_count_steps=64
iterations_per_loop=10000
num_parallel_calls=64

num_train_images=100000
train_batch_size=1024

num_eval_images=9832
eval_batch_size=1024

predict_batch_size=3

use_tpu=False
export_to_tpu=False
transpose_input = True
use_cache=True

#tiny_image_net
input_image_size=64

# Define NAS Model

In [4]:
  tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver('',zone=None, project=None)
  
  config = tf.contrib.tpu.RunConfig(
      cluster=tpu_cluster_resolver,
      model_dir=model_dir,
      save_checkpoints_steps=save_checkpoints_steps,
      log_step_count_steps=log_step_count_steps,
      session_config=tf.ConfigProto(
          graph_options=tf.GraphOptions(
              rewrite_options=rewriter_config_pb2.RewriterConfig(
                  disable_meta_optimizer=True))),
      tpu_config=tf.contrib.tpu.TPUConfig(
          iterations_per_loop=iterations_per_loop,
          per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig
          .PER_HOST_V2))  # pylint: disable=line-too-long
  # Initializes model parameters.
  params = dict(steps_per_epoch=num_train_images / train_batch_size)
  nas_est = tf.contrib.tpu.TPUEstimator(
      use_tpu=use_tpu,
      model_fn=nas_model_fn,
      config=config,
      train_batch_size=train_batch_size,
      eval_batch_size=eval_batch_size,
      predict_batch_size=predict_batch_size,
      export_to_tpu=export_to_tpu,
      params=params)

INFO:tensorflow:Using config: {'_model_dir': '/Users/roxanefischer/Desktop/models_tiny_imagenet_v2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 10000, '_save_checkpoints_secs': None, '_session_config': graph_options {
  rewrite_options {
    disable_meta_optimizer: true
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': None, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': b'', '_evaluation_master': b'', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=10000, num_shards=None, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None), '_cluster': <tensorflow.contrib.

# Load datasets

In [5]:
imagenet_train, imagenet_eval = [
    imagenet_input.ImageNetInput(
        is_training=is_training,
        data_dir=data_dir,
        transpose_input=transpose_input,
        cache=use_cache and is_training,
        image_size=input_image_size,
        num_parallel_calls=num_parallel_calls,
        use_bfloat16=False) for is_training in [True, False]
]

# Evaluation on train and test sets

In [6]:
eval_steps = num_eval_images // eval_batch_size
def eval_results(ckpt ="/Users/roxanefischer/Desktop/models_tiny_imagenet_v2/model.ckpt-10000", eval_steps = eval_steps):
    eval_results = nas_est.evaluate(
        input_fn=imagenet_eval.input_fn,
        steps=eval_steps,
        checkpoint_path=ckpt)
    return eval_results

train_steps = num_train_images // train_batch_size
def train_results(ckpt ="/Users/roxanefischer/Desktop/models_tiny_imagenet_v2/model.ckpt-10000", train_steps = train_steps):
    train_results = nas_est.evaluate(
        input_fn=imagenet_train.input_fn,
        steps=train_steps,
        checkpoint_path=ckpt)
    return train_results

# Accuracies (Useless : not well trained)

## NAS Search

## Checkpoint 10000 on TinyImagenet

### Eval results

{ALL global_step = 10000, loss = 407.13586, **top_1_accuracy = 0.006184896, top_5_accuracy = 0.024631076**}

### Train results 

{ALL global_step = 10000, loss = 403.2064, **top_1_accuracy = 0.35014296, top_5_accuracy = 0.60899246**}


## Checkpoint 785 on TinyImagenet

### Eval results

{1  : global_step = 785, loss = 498.46667, **top_1_accuracy = 0.0048828125, top_5_accuracy = 0.0234375**}

{All :global_step = 785, loss = 498.53845, **top_1_accuracy = 0.0046657985, top_5_accuracy = 0.025282118**}

### Train results 

{1 : global_step = 785, loss = 496.75244, **top_1_accuracy = 0.022460938, top_5_accuracy = 0.064453125**}

{All : global_step = 785, loss = 496.75757, top_1_accuracy = 0.017386839, top_5_accuracy = 0.055271424}



### 1 eval_step on eval_dataset model ckpt = 785
{'loss': 498.56723,
 'top_1_accuracy': 0.005859375,
 'top_5_accuracy': 0.025390625,
 'global_step': 785}

 {'loss': 498.4192,
 'top_1_accuracy': 0.0068359375,
 'top_5_accuracy': 0.025390625,
 'global_step': 785}

{'loss': 498.5098,
 'top_1_accuracy': 0.0029296875,
 'top_5_accuracy': 0.02734375,
 'global_step': 785}

{'loss': 498.39905,
 'top_1_accuracy': 0.005859375,
 'top_5_accuracy': 0.024414062,
 'global_step': 785}

 

# Predict (Useless : not well trained)

In [7]:
%pycache
predictions = nas_est.predict(input_fn=imagenet_eval.input_fn,checkpoint_path=ckpt,yield_single_examples=True)
c=0
for i in predictions:
    c=c+1
    proba = i['probabilities']
    if (c>3):
        break

64, 4, 1, 480)
INFO:tensorflow:DWConv: single-path-search/single_path_super_net/mnas_blocks_10/Relu_1:0 shape: (64, 4, 1, 480)
INFO:tensorflow:Project: single-path-search/single_path_super_net/mnas_blocks_10/Add:0 shape: (64, 4, 1, 80)
INFO:tensorflow:Block input: single-path-search/single_path_super_net/mnas_blocks_10/Add:0 shape: (64, 4, 1, 80)
INFO:tensorflow:Expand: single-path-search/single_path_super_net/mnas_blocks_11/Relu:0 shape: (64, 4, 1, 480)
INFO:tensorflow:DWConv: single-path-search/single_path_super_net/mnas_blocks_11/Relu_1:0 shape: (64, 4, 1, 480)
INFO:tensorflow:Project: single-path-search/single_path_super_net/mnas_blocks_11/Add:0 shape: (64, 4, 1, 80)
INFO:tensorflow:Block input: single-path-search/single_path_super_net/mnas_blocks_11/Add:0 shape: (64, 4, 1, 80)
INFO:tensorflow:Expand: single-path-search/single_path_super_net/mnas_blocks_12/Relu:0 shape: (64, 4, 1, 480)
INFO:tensorflow:DWConv: single-path-search/single_path_super_net/mnas_blocks_12/Relu_1:0 shape: (

In [17]:

result = nlargest(5, enumerate(proba), itemgetter(1))
result

[(59, 0.26125696),
 (141, 0.2529375),
 (132, 0.041710857),
 (118, 0.03478103),
 (149, 0.033681687)]

In [11]:
%pycache
train_results_v2_0 =train_results(ckpt ="/Users/roxanefischer/Desktop/tiny_imagenet_local/v2/model.ckpt-0", train_steps = 10)
train_results_v2_200 =train_results(ckpt ="/Users/roxanefischer/Desktop/tiny_imagenet_local/v2/model.ckpt-200", train_steps = 10)
train_results_v2_400 =train_results(ckpt ="/Users/roxanefischer/Desktop/tiny_imagenet_local/v2/model.ckpt-400", train_steps = 10)

train_results_v3_0 =train_results(ckpt ="/Users/roxanefischer/Desktop/tiny_imagenet_local/v3/model.ckpt-0", train_steps = 10)
train_results_v3_200 =train_results(ckpt ="/Users/roxanefischer/Desktop/tiny_imagenet_local/v3/model.ckpt-200", train_steps = 10)
train_results_v3_400 =train_results(ckpt ="/Users/roxanefischer/Desktop/tiny_imagenet_local/v3/model.ckpt-400", train_steps = 10)

UsageError: Line magic function `%pycache` not found.
